<a href="https://colab.research.google.com/github/jatinscodingproject/loan-chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import indian
import bs4 as bs
import warnings
import urllib.request
import nltk
import random
import string
import datetime
import re

In [ ]:
# Uncomment these below if being run for the first time
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('indian')
nltk.corpus.indian.words('hindi.pos')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package indian to /root/nltk_data...
[nltk_data]   Unzipping corpora/indian.zip.


['पूर्ण', 'प्रतिबंध', 'हटाओ', ':', 'इराक', 'संयुक्त', ...]

In [ ]:
# to filter warnings
warnings.filterwarnings('ignore')

In [ ]:
# inputs for greeting
greeting_inputs = ['हाय', 'नमस्ते']
# inputs for a normal conversation
convo_inputs = ['कैसे हो','अच्छे हो']
# greeting responses by the bot
greeting_responses = ['नमस्ते! मैं आपकी कैसे मदद कर सकता हूँ?',
                      'सुनो! तो आप क्या जानना चाहते हैं?',
                      'नमस्कार, आप मुझसे ऋण के संबंध में कुछ भी पूछ सकते हैं।']
# conversation responses by the bot
convo_responses = ['मैं ठीक हूँ आप कैसे हैं']
# converstional response by the user
convo_replies = ['मैं अपने लोन के बारे में जानना चाहता हूँ']
# few limited questions and answers given as dictionary
question_answers = {'आप क्या हो': 'मैं सहयोग हूँ, लोन सहायक',
                    'आप कौन हो': 'मैं सहयोग हूँ, लोन सहायक',
                    'तुम क्या हो': 'मैं सहयोग हूँ, लोन सहायक',
                    'तुम कौन हो': 'मैं सहयोग हूँ, लोन सहायक',
                    'आप क्या कर सकते हो': 'लोन संबंधी प्रश्नों के उत्तर दें!',
                    'तुम कर सकते हो': 'लोन संबंधी प्रश्नों के उत्तर दें!',
                    'आप को किसने बनाया है': 'मुझे Anurag , Vikas , Jatin  ने बनाया है',
                    'लोन क्या है':'एक लोन एक संपत्ति, धन, या अन्य भौतिक सामान है जो किसी अन्य पार्टी को लोन मूल्य के भविष्य के पुनर्भुगतान के साथ-साथ ब्याज और अन्य वित्त शुल्क के बदले में दिया जाता है।',
                    'व्यक्तिगत लोन क्या है':'एक व्यक्तिगत लोन (उपभोक्ता लोन के रूप में भी जाना जाता है) किसी भी स्थिति का वर्णन करता है जिसमें एक व्यक्ति व्यक्तिगत जरूरतों के लिए धन उधार लेता है, जिसमें कंपनी में निवेश करना शामिल है। सभी व्यक्तिगत ऋणों में तीन सामान्य तत्व होते हैं: ऋण का साक्ष्य (वचन पत्र) उधार ली गई राशि (मूलधन)',
                    'वाहन लोन क्या है':'वाहन लोन एक ऐसा लोन है जो आपको व्यक्तिगत उपयोग के लिए दो और चार पहिया वाहन खरीदने की अनुमति देता है। आमतौर पर, ऋणदाता धन को लोन देता है (खरीदार की ओर से डीलर को सीधे भुगतान करता है) जबकि खरीदार को एक विशिष्ट ब्याज दर पर एक विशिष्ट अवधि में समान मासिक किश्तों (ईएमआई) में ऋण चुकाना होता है',
                    'घर लोन क्या है':'घर खरीद लोन का उपयोग आवासीय संपत्ति खरीदने के लिए किया जाता है। ऋणदाता आमतौर पर 80-85% तक प्रदान करते हैंमंडी लोन राशि के रूप में घर का मूल्य। ऋणों की ब्याज दर या तो स्थिर, अस्थायी या संकर होती है।',
                    'व्यक्तिगत लोन की पात्रता क्या है':'आयु: 21 वर्ष से 67 वर्ष*. सार्वजनिक, निजी, या एमएनसी कंपनी के साथ नौकरी. सिबिल स्कोर: 750 या उससे अधिक. मासिक सेलरी: कम से कम रु.25000',
                    'घर लोन की पात्रता क्या है':'घर लोन की योग्यता आय की स्थिरता पर निर्भर करती है व्यवसाय या व्यवसाय की प्रकृति लोन की तलाश करने वाले की उम्र क्रेडिट स्कोर संपत्ति के गुण बैंक और कंपनी के साथ आपका संबंध है जो आप काम करते हैं। आमतौर पर, सभी बैंक आपके मासिक नेट के 60 गुने तक घर ऋण प्रदान करते हैं आय',
                    'वाहन लोन की पात्रता क्या है':'क्तियों के पास मासिक होना चाहिएआय 20000 रुपये का अधिक। आपने अपने वर्तमान नियोक्ता के साथ रोजगार का 1 वर्ष पूरा कर लिया होगा। आवेदक व्यक्तियों को इस तरह से नियोजित किया जाना चाहिए कि इसे सरकार में नियोजित किया जा सकेसुविधा, एक निजी कंपनी में या यहां तक कि स्वरोजगार में।',
                    'घर लोन अप्लाई करने के लिए किन दस्तावेजों की जरूरत होती है':'''होम लोन के लिए आवश्यक डॉक्यूमेंट की लिस्ट \n
                                                                             1.प्रॉपर्टी डॉक्यूमेंट \n
                                                                             2.सेल्स डीड, बिक्री का स्टाम्प्ड एग्रीमेंट, या एलॉटमेंट लेटर \n
                                                                             3.हाउसिंग सोसाइटी या बिल्डर से NOC \n
                                                                             4.भूमि/भूमि राजस्व/राजस्व विभाग के लिए कब्जा प्रमाणपत्र और भूमि कर रसीद \n
                                                                             5.निर्माण लागत का विस्तृत आकलन''',
                    'व्यक्तिगत लोन अप्लाई करने के लिए किन दस्तावेजों की जरूरत होती है':''' पर्सनल लोन/ व्यक्तिगत ऋण एक प्रकार का असुरक्षित लोन है जो किसी योग्य आवेदक को उसकी फाइनेंशियल आवश्यकताओं को पूरा करने के लिए दिया जाता है। \n
                                                                             1.पहचान का प्रमाण (निम्नलिखित में से कोई एक): \n
                                                                             2.पैन कार्ड \n
                                                                             3.पासपोर्ट \n
                                                                             3.ड्राइविंग लाइसेंस \n
                                                                             4.मतदाता पहचान पत्र \n
                                                                             5.कर्मचारी आईडी \n
                                                                             5.बैंक पासबुक \n
                                                                             7.राशन पत्रिका''',
                    'वाहन लोन अप्लाई करने के लिए किन दस्तावेजों की जरूरत होती है':'''कार लोन के आवेदन के लिए किन दस्तावेजों की जरूरत होती है ? \n
                                                                             1.कार लोन के आवेदन के लिए आपको निम्नलिखित दस्तावेज देने होंगे \n
                                                                             2.पहचान का प्रमाण \n
                                                                             3.पते का प्रमाण \n
                                                                             4.केवाईसी दस्तावेज \n
                                                                             5.आय का प्रमाण (फॉर्म 16 या 6 माह की वेतन स्लिप) \n
                                                                             6.उम्र का प्रमाण (न्यूनतम 18 साल) \n
                                                                             7.फोटोग्राफ \n
                                                                             8.कार के कागजात''',
                    'ghar लोन कैसे अप्लाई करें':'कृपया हमारी वेबसाइट www.sahyogfinance.com पर जाएं और गृह लोन चुनें और फॉर्म भरें और हमारी समीक्षा के लिए 24 घंटे तक प्रतीक्षा करें।',
                    'वाहन लोन कैसे अप्लाई करें':'कृपया हमारी वेबसाइट www.sahyogfinance.com पर जाएं और वाहन लोन चुनें और फॉर्म भरें और हमारी समीक्षा के लिए 24 घंटे तक प्रतीक्षा करें',
                    'व्यक्तिगत लोन कैसे अप्लाई करें':'कृपया हमारी वेबसाइट www.sahyogfinance.com पर जाएं और व्यक्तिगत लोन चुनें और फॉर्म भरें और हमारी समीक्षा के लिए 1 घंटे तक प्रतीक्षा करें',
                    'राशि का भुगतान करने में कितना समय लगेगा':'अंतिम स्वीकृति प्राप्त करने में आपके पंजीकृत बैंक में राशि का भुगतान करने में 1 घंटा लगता है।',
                    'आप क्या करते हैं': 'ऋण संबंधी प्रश्नों के उत्तर दें!',
                    'क्या मुझे अपने ऋण पर विस्तार मिल सकता है': 'हाँ यदि आप पात्र हैं तो आपको मिलेगा',
                    'मैं 50000 उधार लेना चाहता हूं इसके लिए कौन से दस्तावेज आवश्यक हैं': 'आपको आय प्रमाण, आधार कार्ड, पैन नंबर, पुनर्वितरण प्रमाण देना होगा'}

In [ ]:
#Define loan product and repayment data
loan_products = {'1': '''20 - 40 वर्ष :- ब्याज दर: 8.5%  अधिभुत अवधि: 60 महीने \n
                         40 - 45 वर्ष :- ब्याज दर: 9.5% अधिभुत अवधि: 48 महीने  \n
                         45 - 55 वर्ष :- ब्याज दर: 10.5% अधिभुत अवधि: 36 महीने''',
                 '2': '''20 - 40 वर्ष :- ब्याज दर: 4.5%  अधिभुत अवधि: 60 महीने \n
                         40 - 45 वर्ष :- ब्याज दर: 4.5% अधिभुत अवधि: 48 महीने \n
                         45 - 55 वर्ष :- ब्याज दर: 4.5% अधिभुत अवधि: 48 महीने''',
                 '3': '''20 - 40  वर्ष :- ब्याज दर: 8.5% अधिभुत अवधि: 240 महीने \n
                         40 - 45 वर्ष :- ब्याज दर: 9.5% अधिभुत अवधि: 180 महीने  \n
                         45 - 50 वर्ष :- ब्याज दर: 10.5% अधिभुत अवधि: 120 महीने''',
                 '4': '''20 - 40  वर्ष :- ब्याज दर: 8.5% अधिभुत अवधि: 60 महीने \n
                         40 - 45 वर्ष :- ब्याज दर: 9.5% अधिभुत अवधि: 60 महीने \n
                         45 - 50 वर्ष :- ब्याज दर: 10.5% अधिभुत अवधि: 48 महीने''',
                 '6': 'कृपया इस वेबसाइट www.sahyogfinance.com पर जाएं और लोन के लिए आवेदन करें',
                 '7': 'पुनर्भुगतान लिंक आपको मेल पर भेजा जाएगा'}

repayment_data = {'देर से भुगतान के लिए जुर्माना': 500 , 'छूटे भुगतान के लिए जुर्माना': 150, 'भुगतान अंतराल': 'मासिक'}
#Define customer data
repayment_input = ['5','ऋण बकाया राशि']
customer_data = {'100001': {'ऋण उत्पाद': 'व्यक्तिगत ऋण', 'ऋण राशि': 10000, 'बकाया राशि': 6500, 'भुगतान इतिहास': 'टाइम पर', 'कार्यकाल':3,'ईएमआई तिथि':'हर महीने की 2 तारीख','नियत तारीख':[10,3,2023],'किस्त राशि':2500},
                 '100002': {'ऋण उत्पाद': 'कार ऋण', 'ऋण राशि': 200000, 'बकाया राशि': 40000, 'भुगतान इतिहास': 'टाइम पर','कार्यकाल':60,'ईएमआई तिथि':'हर महीने की 5 तारीख','नियत तारीख':[12,4,2023],'किस्त राशि':3000},
                 '100003': {'ऋण उत्पाद': 'घर ऋण', 'ऋण राशि': 5000000, 'बकाया राशि': 4500000, 'भुगतान इतिहास': 'हमेशा देर से','कार्यकाल':240,'ईएमआई तिथि':'हर महीने की 2 तारीख','नियत तारीख':[12,4,2023],'किस्त राशि':2800},
                 '100004': {'ऋण उत्पाद': 'व्यक्तिगत ऋण', 'ऋण राशि': 50000, 'बकाया राशि': 25000, 'भुगतान इतिहास': 'टाइम पर','कार्यकाल':10,'ईएमआई तिथि':'हर महीने की 5 तारीख','नियत तारीख':[12,4,2023],'किस्त राशि':1700}}
# define enquiry response
inquiry_response = ['6','नए आवेदन']

In [ ]:
repayment_reminder_response = "{payment_amount:.2f} का भुगतान {due_date} तक करना होगा।"
loan_inquiry_response = "हम आपको {loan_product} ऋण राशि {loan_amount} के लिए ऋण की जानकारी दे सकते हैं।"

In [ ]:
# fetching raw html data about brac from wiki
raw_data = urllib.request.urlopen('https://hi.wikipedia.org/wiki/%E0%A4%8B%E0%A4%A3#:~:text=%E0%A4%8B%E0%A4%A3%20%E0%A4%B5%E0%A4%B9%20%E0%A4%B9%E0%A5%88%2C%20%E0%A4%9C%E0%A5%8B%20%E0%A4%95%E0%A4%BF%E0%A4%B8%E0%A5%80,%E0%A4%95%E0%A5%87%20%E0%A4%AA%E0%A5%8D%E0%A4%B0%E0%A4%AF%E0%A5%8B%E0%A4%97%20%E0%A4%95%E0%A4%BE%20%E0%A4%AE%E0%A4%BE%E0%A4%A7%E0%A5%8D%E0%A4%AF%E0%A4%AE%20%E0%A4%B9%E0%A5%88%E0%A5%A4)')
# processing the raw html into more readable data
raw_data = raw_data.read()

In [ ]:
# turning html into text
article = bs.BeautifulSoup(raw_data, 'lxml')

In [ ]:
# extracting paras from the above xml and concatenating with article_text
paragraphs = article.find_all('p')

article_text = ''

for p in paragraphs:
    article_text += p.text

article_text = article_text.lower()

In [ ]:
# getting rid of all the special characters
article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
article_text = re.sub(r'\s+', ' ', article_text)

In [ ]:
# extracting sentences from the article
sentences = nltk.sent_tokenize(article_text)

In [ ]:
# extracting words from the article
words = nltk.word_tokenize(article_text)

In [ ]:
lemma = nltk.stem.WordNetLemmatizer()

In [ ]:
# lemmatizing words as a part of pre-processing
def perform_lemmatization(tokens):
    return [lemma.lemmatize(token) for token in tokens]

In [ ]:
# removing punctuation
remove_punctuation = dict((ord(punc), None) for punc in string.punctuation)

In [ ]:
# method to pre-process all the tokens utilizing the above functions
def processed_data(document):
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(remove_punctuation)))

In [ ]:
# function for punctuation removal
def punc_remove(str):
    punctuations = r'''!()-[]{};:'"\,<>./?@#$%^&*_~\n'''
    no_punct = ''

    for char in str:
        if char not in punctuations:
            no_punct = no_punct + char

    return no_punct

In [ ]:
# Define function to handle loan inquiries
def handle_loan_inquiry(loan_products, loan_amount):
    monthly_payment, total_interest, total_payment = calculate_repayment_details(loan_products, loan_amount)
    response = loan_inquiry_response.format(loan_product=loan_products, loan_amount=loan_amount, monthly_payment=monthly_payment, total_interest=total_interest, total_payment=total_payment)
    return response

In [ ]:
# Define function to calculate loan repayment details
def calculate_repayment_details(loan_products, loan_amount):
    interest_rate = loan_products[loan_products]['ब्याज दर']
    loan_term = loan_products[loan_products]['अधिभुत अवधि']
    monthly_interest_rate = interest_rate / 12
    monthly_payment = (loan_amount * monthly_interest_rate * (1 + monthly_interest_rate) ** loan_term) / ((1 + monthly_interest_rate) ** loan_term - 1)
    total_interest = monthly_payment * loan_term - loan_amount
    total_payment = loan_amount + total_interest
    return monthly_payment, total_interest, total_payment

In [ ]:
# Define function to handle loan repayment reminders
def handle_repayment_reminder(loan_no):
    loan_details = customer_data.get(loan_no)
    day, month, year  = loan_details.get('नियत तारीख')
    due_date = datetime.date(year, month, day)
    current_date = datetime.date.today()
    penalty=0
    penalty_amount=0.0
    if current_date > due_date:
        days_late = (current_date-due_date).days
        penalty = days_late * repayment_data['देर से भुगतान के लिए जुर्माना']
    instalment_amount = loan_details['किस्त राशि']
    if penalty>0:
      penalty_amount = (instalment_amount*penalty)/100
      instalment_amount = instalment_amount + penalty_amount
    due_date_str = str(due_date) + ' तक'
    response = repayment_reminder_response.format(payment_amount=instalment_amount, due_date=due_date_str)
    return response

In [ ]:
# method to generate a loan enquiry
def generate_enquiry_response(str):
    if punc_remove in inquiry_response:
       amount = int(input("आप कितना उधार लेना चाहते हैं"))
       name = str("आपका क्या नाम है ?")
       print('नमस्ते' + name)
       product = str(input('''आप कौन सा उत्पाद लेना चाहते हैं? \n 1:व्यक्तिगत ऋण \n 2 : कृषि ऋण \n 3 : गृह ऋण \n 4 : वाहन ऋण'''))
       amount = int(input("आप कितना उधार लेना चाहते हैं"))
       aadhar_no = str(input("कृपया अपना आधार नंबर प्रदान करें"))
       pan_no = str(input("कृपया अपना पैन नंबर प्रदान करें"))
       return "हमने आपकी पूछताछ को संग्रहीत कर लिया है हम शीघ्र ही आपसे संपर्क करेंगे।"

In [ ]:
# method to generate a response to greetings
def generate_greeting_response(str):
    if punc_remove in greeting_inputs:
        return random.choice(greeting_responses)

In [ ]:
# method to generate a response to conversations
def generate_convo_response(str):
    if punc_remove(str) in convo_inputs:
        return random.choice(convo_responses)

In [ ]:
# method to generate a answers to questions
def generate_answers(str):
    if punc_remove(str) in question_answers:
        return question_answers[punc_remove(str)]

In [ ]:
# method to generate a loan product response
def generate_loan_products_response(str):
    if punc_remove(str) in loan_products:
        return loan_products[punc_remove(str)]


In [ ]:
def generate_repayment_response(str):
    if punc_remove(str) in repayment_input:
       return "कृपया अपना ऋण संख्या प्रदान करें।"
    elif str.isdigit():
       return handle_repayment_reminder(str)

In [ ]:
# method to generate response regarding loan enquiries
def generate_response(str):
    Sahyog_response = ''
    sentences.append(str)

    word_vectorizer = TfidfVectorizer(tokenizer=processed_data, stop_words='english')
    all_word_vectors = word_vectorizer.fit_transform(sentences)
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
    similar_sentence_number = similar_vector_values.argsort()[0][-2]

    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched is 0:
        Sahyog_response = Sahyog_response +'क्षमा करें, मेरे डेटाबेस में ऐसा जानकारी नहीं है। कोशिश कुछ अलग और कर्ज से जुड़ा है।'
        return Sahyog_response
    else:
        Sahyog_response = Sahyog_response + sentences[similar_sentence_number]
        return Sahyog_response

In [ ]:
# chatting with the chatbot -->
def run_chatbot(user_input):
        if user_input != '8':
            if user_input == 'धन्यवाद' or user_input == 'आपका बहुत-बहुत धन्यवाद' or user_input == 'धन्यवाद':
                return 'मेरे साथ बात करने के लिए धन्यवाद'
            elif user_input in convo_replies:
                return 'यह अच्छा है! मैं लोन की सहायता कैसे कर सकता हूं?'
            else:
                if generate_greeting_response(user_input) is not None:
                    return generate_greeting_response(user_input)
                elif generate_convo_response(user_input) is not None:
                    return generate_convo_response(user_input)
                elif generate_answers(user_input) is not None:
                    return generate_answers(user_input)
                elif generate_loan_products_response(user_input) is not None:
                    return generate_loan_products_response(user_input)
                elif generate_repayment_response(user_input) is not None:
                    return generate_repayment_response(user_input)
                elif generate_enquiry_response(user_input) is not None:
                    return generate_enquiry_response(user_input)
                else:
                    return generate_response(user_input)
        else:
             return 'सहयोग धन्यवाद, जल्द मिलते हैं।'

In [ ]:
pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=5c50b312fbb834d974c4443e8b2ac776c37c1e742d28f0cd516ceb5e358eb0b2
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
pip install flask-ngrok

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/static

/content/drive/MyDrive/static


In [ ]:
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
from flask import Flask, render_template, request
import numpy as np

In [ ]:
ngrok.set_auth_token('2VG0TchODb5QjlNRf15x2hi5sbc_Ngpwbrfkn4ZwCrrecTc')

In [ ]:
public_url = ngrok.connect(5000).public_url

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)
@app.route("/")
def home():
    return render_template("chatbot.html")
@app.route("/get")
def get_bot_response():
    user_input = request.args.get('msg')
    return run_chatbot(user_input)
app.run()
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://a7a4-34-68-36-181.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


ERROR:__main__:Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-37-4c48c3415658>", line 5, in home
    return render_template("chatbot.html")
  File "/usr/local/lib/python3.10/dist-packages/flask/templating.py", line 146, in render_template
    template = app.jinja_env.get_or_select_template(template_name_or_list)
  File "/usr/local/lib/python3.10/dist-packages/jinja2/environment.py", line 1081, 